In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import path
import pandas as pd
import sys
import json
import torch
from torch.utils.data import DataLoader
import lightning.pytorch as pl
from lightning.pytorch.callbacks import ModelCheckpoint
import os
import wandb
from argparse import Namespace
import datetime

import util as tu
import model as tm
from setup import *


c:\Users\TanJI\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\TanJI\AppData\Local\Programs\Python\Python310\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")
c:\Users\TanJI\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
configs = Namespace (
    project_name = "TanJI",
    test_database_ids = [13],
    random_seed = 114514,
    batch_size = 1000,
    accelerator = 'cpu',
    progress_bar = True,
    lr = 0.001,
    dropout = 0.2,
    epochs = 100,
    pad_length = 20,
    node_length = 19,
    hidden_dim = 192,
    output_dim = 1,
    newpath = './results/full/cost/',
    to_predict = 'cost',
    dataset_file_name = 'alldataset.pkl',
)

In [4]:
with open(path.join(ROOT_DIR, 'data', 'workload1', 'statistics.json')) as file:
    feature_statistics = json.load(file)
# with open(path.join(ROOT_DIR, 'data', 'imdb', 'statistics.json')) as file:
#     feature_statistics = json.load(file)
configs.node_length = len(feature_statistics['node_types']['value_dict']) + 2
encoder = tu.Encoder(feature_statistics, configs)

{'Index Only Scan': 0, 'Index Scan': 1, 'Seq Scan': 2, 'Hash Join': 3, 'Bitmap Index Scan': 4, 'Result': 5, 'Merge Join': 6, 'Limit': 7, 'Aggregate': 8, 'Sort': 9, 'Hash': 10, 'Bitmap Heap Scan': 11, 'Nested Loop': 12, 'Materialize': 13, 'BitmapOr': 14, 'Memoize': 15, 'Gather': 16, 'Gather Merge': 17, 'BitmapAnd': 18}
{'Actual Total Time': RobustScaler(), 'Plan Rows': RobustScaler(), 'Total Cost': RobustScaler()}


In [5]:
import pickle

# 存储类的实例
def save_instance(instance, filename):
    with open(filename, 'wb') as file:
        pickle.dump(instance, file)

# 加载存储的实例
def load_instance(filename):
    with open(filename, 'rb') as file:
        instance = pickle.load(file)
    return instance

In [6]:
# for zero-shot data
files_name = []

for wk_item in workloads:
    files_name.append(path.join(ROOT_DIR, 'data', 'workload1', wk_item + '_filted.json'))
# for i in range(20):
#     files_name.append(path.join(ROOT_DIR, 'data', 'imdb', 'plan_and_cost', 'train_plan_part{}.csv'.format(i)))

data_file_name = path.join(ROOT_DIR, 'data', 'workload1', configs.dataset_file_name)
# data_file_name = path.join(ROOT_DIR, 'data', 'imdb', configs.dataset_file_name)

if os.path.exists(data_file_name):
    alldataset = load_instance(data_file_name)
else:
    dict_list = tu.Encoder.format_workload(files_name)
    # dict_list = tu.Encoder.format_imdb(files_name)
    alldataset = tu.PlanTreeDataSet(dict_list, encoder)
    save_instance(alldataset, data_file_name)

torch.Size([200000, 1, 420]) torch.Size([200000, 20, 20]) torch.Size([200000, 20]) torch.Size([200000, 20])


In [7]:
train_dataloader, val_dataloader, test_dataloader = tu.get_dataloader(alldataset, configs)

5 4 4


In [8]:
nowtime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
model = tm.TanJI(configs)
logger = pl.loggers.WandbLogger(project=configs.project_name, name=nowtime)
logger.log_hyperparams(configs.__dict__)
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    dirpath=os.path.join(ROOT_DIR, "checkpoints"),
    filename="TanJI",
)
trainer = tm.PLTrainer(
    accelerator=configs.accelerator,
    enable_progress_bar=configs.progress_bar,
    enable_model_summary=configs.progress_bar,
    max_epochs=configs.epochs,
    logger=logger,
    callbacks=[checkpoint_callback],
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tanji. Use `wandb login --relogin` to force relogin


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [9]:
# trainer = PLTrainer(accelerator="cpu", max_epochs=50, logger=wandb_logger)
trainer.fit(model, train_dataloader, val_dataloader)

c:\Users\TanJI\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:617: UserWarning: Checkpoint directory D:\TanJI\Documents\document\Work\Research\DB4AI\2024届本科毕业论文（设计）工作指南(1)\TanJI\checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name               | Type               | Params
----------------------------------------------------------
0 | tranformer_encoder | TransformerEncoder | 10.2 K
1 | mlp                | Sequential         | 62.2 K
2 | sigmoid            | Sigmoid            | 0     
----------------------------------------------------------
72.4 K    Trainable params
0         Non-trainable params
72.4 K    Total params
0.290     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\TanJI\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:490: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(
c:\Users\TanJI\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\TanJI\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\activation.py:1160: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered interna

c:\Users\TanJI\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 99: 100%|██████████| 171/171 [00:28<00:00,  5.98it/s, v_num=5gb8]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 171/171 [00:28<00:00,  5.98it/s, v_num=5gb8]


In [10]:
# test model
result = trainer.test(model, dataloaders=test_dataloader)